In [1]:
import pandas as pd

In [2]:
codes = pd.read_csv('MIMIC/cleaned_data_full.csv').dropna()

In [3]:
from collections import defaultdict
import csv
def reformat(code, is_diag):
    code = ''.join(code.split('.'))
    if is_diag:
        if code.startswith('E'):
            if len(code) > 4:
                code = code[:4] + '.' + code[4:]
        else:
            if len(code) > 3:
                code = code[:3] + '.' + code[3:]
    else:
        code = code[:2] + '.' + code[2:]
    return code

def load_code_descriptions(version='mimic3'):
    desc_dict = defaultdict(str)
    DATA_DIR = '../caml-mimic/mimicdata'
    with open("%s/D_ICD_DIAGNOSES.csv" % (DATA_DIR), 'r') as descfile:
        r = csv.reader(descfile)
        next(r)
        for row in r:
            code = row[1]
            desc = row[-1]
            desc_dict[reformat(code, True)] = desc
            
    with open("%s/D_ICD_PROCEDURES.csv" % (DATA_DIR), 'r') as descfile:
        r = csv.reader(descfile)
        next(r)
        for row in r:
            code = row[1]
            desc = row[-1]
            if code not in desc_dict.keys():
                desc_dict[reformat(code, False)] = desc
                
    with open('%s/ICD9_descriptions' % DATA_DIR, 'r') as labelfile:
        for i,row in enumerate(labelfile):
            row = row.rstrip().split()
            code = row[0]
            if code not in desc_dict.keys():
                desc_dict[code] = ' '.join(row[1:])
    return desc_dict

In [4]:
desc_dict = load_code_descriptions()

In [5]:
for k in desc_dict :
    if '285' in k : print(k, desc_dict[k])

285.0 Sideroblastic anemia
285.1 Acute posthemorrhagic anemia
285.21 Anemia in chronic kidney disease
285.22 Anemia in neoplastic disease
285.29 Anemia of other chronic disease
285.3 Antineoplastic chemotherapy induced anemia
285.8 Other specified anemias
285.9 Anemia, unspecified
285 Other and unspecified anemias
285.2 Anemia of chronic disease


In [6]:
def has_label(x, positive_icd9) :
    return any([positive_icd9 in y for y in x.split(';')])

In [7]:
codes['has_c1'] = codes['LABELS'].apply(lambda x : has_label(x, '285.1'))
codes['has_c2'] = codes['LABELS'].apply(lambda x : has_label(x, '285.2'))

In [8]:
codes.groupby(['has_c1', 'has_c2']).size()

has_c1  has_c2
False   False     45661
        True       2562
True    False      4283
        True        216
dtype: int64

In [9]:
data_for_pos_label = codes[(codes['has_c1'] == True) & (codes['has_c2'] == False)]
data_for_neg_label = codes[(codes['has_c2'] == True) & (codes['has_c1'] == False)]

In [10]:
data = pd.concat([data_for_neg_label, data_for_pos_label]).reset_index(drop=True)

In [11]:
import spacy, re
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

def cleaner(text, spacy=True) :
    text = re.sub(r'\s+', ' ', text.strip())
    if spacy :
        text = [t.text.lower() for t in nlp(text)]
    else :
        text = [t.lower() for t in text.split()]
    text = " ".join(text)
    text = re.sub(r'\[\s*\*\s*\*(.*?)\*\s*\*\s*\]', ' <DE> ', text)
    text = re.sub(r'([^a-zA-Z0-9])(\s*\1\s*)+', r'\1 ', text)
    text = re.sub(r'\s+', ' ', text.strip())
    text = ['qqq' if any(char.isdigit() for char in word) else word for word in text.split(' ')]
    return " ".join(text)

In [12]:
from tqdm import tqdm_notebook

In [13]:
for k in desc_dict :
    desc_dict[k] = cleaner(desc_dict[k])

In [16]:
data.to_csv('/media/sarthak/data/projects/Transparency/preprocess/MIMIC/cleaned_data' + 'anemia' + '.csv', index=False)

In [17]:
import vectorizer as v
Vectorizer = v.Vectorizer

In [18]:
vec = Vectorizer(min_df=5)

In [19]:
vec.fit(list(data['TEXT']) + list(desc_dict.values()))

In [21]:
vec.label2idx = {k:i for i, k in enumerate(desc_dict.keys())}
vec.idx2label = {i:k for k, i in vec.label2idx.items()}
vec.label2desc = desc_dict
vec.code_desc = {k:vec.convert_to_sequence([x])[0] for k, x in desc_dict.items()}

In [22]:
from sklearn.model_selection import train_test_split
idxs = {}
idxs['train'], idxs['test'] = train_test_split(data.index, stratify=data['has_c1'], test_size=0.2, random_state=12939)

In [23]:
keys = ['train', 'test']
import numpy as np
vec.seq_text = {}
vec.label = {}
for k in keys :
    filtered = data[data.index.isin(idxs[k])]
    vec.seq_text[k] = vec.texts_to_sequences(list(filtered['TEXT']))
    vec.label[k] = np.where(list(filtered['has_c1']), 1, 0)

In [33]:
from gensim.models import KeyedVectors
model = KeyedVectors.load('MIMIC/mimic_embedding_model.wv')

In [35]:
vec.extract_embeddings(model)

Found 19739 words in model out of 19743


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-7.22315192e-01,  4.69720140e-01, -8.95069323e-01, ...,
         8.21053461e-02, -1.43076575e+00,  1.20804264e-01],
       [-2.00530385e-01,  4.51679918e-01, -1.70827172e-02, ...,
         4.37738392e-01,  1.97100196e-01, -1.55563818e-01],
       ...,
       [ 1.16788733e+00, -9.24137831e-01,  1.75671399e+00, ...,
         9.75742284e-03, -2.87074357e-01, -2.45802402e-01],
       [ 9.31499293e-04, -9.84110713e-01, -1.11939907e+00, ...,
        -6.90321016e+00,  3.93742228e+00, -1.51261103e+00],
       [-7.20303178e-01, -1.40906677e-01, -2.27291822e+00, ...,
        -1.95523810e+00,  7.60727704e-01, -1.39611304e+00]])

In [36]:
import pickle
pickle.dump(vec, open('MIMIC/vec_icd9_' + 'anemia' + '.p', 'wb'))